In [1]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns

In [2]:
def TimeDiff(df, frameSize):
    count = 1
    classLabel = df.loc[1,"ClassLabel"]
    time_diff = [0]
    rows = (df.shape)[0]
    for i in range(1,rows):
        if((df.loc[i+1,"ClassLabel"] == classLabel) and (count < frameSize)):
            diff = datetime.datetime.strptime(df.iloc[i,0].split(' ')[3],"%H:%M:%S") - datetime.datetime.strptime(df.iloc[i-1,0].split(' ')[3],"%H:%M:%S")
            time_diff.append(diff.total_seconds())
            count = count + 1
        else:
            time_diff.append(0)
            classLabel = df.loc[i+1,"ClassLabel"]
            count = 1    
    df["SNMP Parameters"] = time_diff
    df.rename(index=str, columns={"SNMP Parameters": "Time Difference"})

In [3]:
def difference(df,cols):
    classLabel = df.loc[1,"ClassLabel"]
    diffList = []

    for j in cols:
        diffList.append([df.loc[1,j]])
        
    rows = (df.shape)[0]
    for i in range(2,rows+1):
        if((df.loc[i,"ClassLabel"] == classLabel)):
            for j in range(len(cols)):
                diffList[j].append(df.loc[i,cols[j]] - df.loc[i-1,cols[j]])
        else:
            for j in range(len(cols)):
                diffList[j].append(df.loc[i,cols[j]])
            classLabel = df.loc[i,"ClassLabel"]
    
    for j in range(len(cols)):
        df[cols[j]] = diffList[j]

In [4]:
def makeNewDataFrame(df, frameSize):
    i = 0
    listOfRowVals = []
    rows = (df.shape)[0]
    while(i+frameSize-1<rows):
        if(df.iloc[i,-1] == df.iloc[i+frameSize-1,-1]):
            rowVals = []
            for j in range(frameSize):
                rowVals = rowVals + list(df.iloc[i+j,:-1])
            rowVals.append(df.iloc[i,-1])
            i = i+frameSize
            listOfRowVals.append(rowVals)
        else:
            j = 1
            while(df.iloc[i,-1] == df.iloc[i+j,-1]):                        
                j=j+1
            i = i+j   
    new_df = pd.DataFrame(listOfRowVals)
    return new_df

In [5]:
df = pd.read_csv('./SNMP_Dataset.csv')
#To drop iso values
df = df[1:]
#Coding Class Types into integers
labelDict = {'ICMP_Normal' : 0, 'UDP_Normal' : 1, 'LAND_Normal' : 2,
            'TCPSYN_Normal' : 3, 'TCPSYNACK_Normal' : 4, 'ICMP_Flood' : 5,
            'UDP_Flood' : 6, 'LAND_Flood' : 7, 'TCPSYN_Flood' : 8,
             'TCPSYNACK_Flood' : 9}
limit = len(df['ClassLabel']) + 1
#df['ClassLabel'] = df['ClassLabel'].replace(['ICMP_Normal','UDP_Normal','LAND_Normal','TCPSYN_Normal','TCPSYNACK_Normal','ICMP_Flood','UDP_Flood','LAND_Flood','TCPSYN_Flood','TCPSYNACK_Flood'],[0,1,2,3,4,5,6,7,8,9])
for i in range(1, limit):
    df['ClassLabel'][i] = labelDict[df['ClassLabel'][i]]   

In [6]:
frameSize = 6
TimeDiff(df, frameSize)
df

,SNMP Parameters,icmp_AddrMaskReps,icmp_InAddrMasks,icmp_InDestUnreachs,icmp_InEchosReps,icmp_InEchos,icmp_InErrors,icmp_InMsgs,icmp_InParmProbs,icmp_InRedirects,...,tcp_PassiveOpens,tcp_RetransSegs,tcp_RtoAlgorithm,tcp_RtoMax,tcp_RtoMin,udp_InDatagrams,udp_InErrors,udp_NoPorts,udp_OutDatagrams,ClassLabel
1,0.0,0,0,0,16,0,0,17,0,0,...,1,0,1,120000,200,148,0,1,149,0
2,33.0,0,0,0,111,0,0,112,0,0,...,2,0,1,120000,200,445,0,1,446,0
3,39.0,0,0,0,229,0,0,230,0,0,...,2,0,1,120000,200,741,0,1,742,0
4,34.0,0,0,0,0,0,0,332,0,0,...,2,0,1,120000,200,1037,0,1,1038,0
5,40.0,0,0,0,442,0,0,443,0,0,...,2,0,1,120000,200,1333,0,1,1334,0
6,32.0,0,0,0,548,0,0,549,0,0,...,2,0,1,120000,200,1629,0,2,1630,0
7,0.0,0,0,0,656,0,0,657,0,0,...,2,0,1,120000,200,1925,0,2,1926,0
8,30.0,0,0,0,766,0,0,767,0,0,...,2,0,1,120000,200,2221,0,2,2222,0
9,41.0,0,0,0,871,0,0,872,0,0,...,2,0,1,120000,200,2517,0,2,2518,0
10,31.0,0,0,0,985,0,0,986,0,0,...,2,0,1,120000,200,2813,0,2,2814,0


In [7]:
df = df.astype(int)
#To drop columns with same values throughout -> 36 columns got dropped
#Dimentionality Reduction
df = df.drop(df.std()[(df.std() == 0)].index, axis=1)    

In [8]:
df.head()

,SNMP Parameters,icmp_InEchosReps,icmp_InErrors,icmp_InMsgs,icmp_InTimeExcds,icmp_InTimestamps,icmp_OutEchoReps,icmp_OutErrors,icmp_OutTimeExcds,icmp_OutTimestamps,...,tcp_EstabResets,tcp_InSegs,tcp_OutRsts,tcp_OutSegs,tcp_PassiveOpens,tcp_RetransSegs,udp_InDatagrams,udp_NoPorts,udp_OutDatagrams,ClassLabel
1,0,16,0,17,1,0,0,17,1,16,...,0,178,0,213,1,0,148,1,149,0
2,33,111,0,112,1,0,0,112,1,127,...,0,198,0,243,2,0,445,1,446,0
3,39,229,0,230,1,0,0,230,1,229,...,0,255,0,282,2,0,741,1,742,0
4,34,0,0,332,1,0,0,347,1,346,...,0,261,0,287,2,0,1037,1,1038,0
5,40,442,0,443,1,0,0,443,1,442,...,0,273,0,296,2,0,1333,1,1334,0


In [9]:
colsToFindDifference=['icmp_InEchosReps','icmp_InMsgs','icmp_OutErrors','icmp_OutTimestamps','ip_InDelivers','ip_InReceives','ip_OutRequests','udp_InDatagrams','udp_OutDatagrams']
difference(df,colsToFindDifference)

In [10]:
df_ = makeNewDataFrame(df, frameSize)
df_.to_csv('datasetToLearn.csv')
df_.head()

,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
0,0,16,0,17,1,0,0,17,1,16,...,0,273,0,296,2,0,296,2,296,0
1,0,108,0,108,1,0,0,91,1,91,...,0,273,0,296,2,0,296,2,296,0
2,0,98,0,98,1,0,0,82,1,97,...,0,299,0,316,2,0,296,2,296,0
3,0,93,0,93,1,0,0,93,1,108,...,0,299,0,316,2,0,296,2,296,0
4,0,104,0,104,1,0,0,2618,1,2617,...,0,299,0,316,2,0,296,2,296,0
